In [8]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob


# re-size all the images to this
IMAGE_SIZE = [100, 100] 

# training config:
epochs = 5
batch_size = 32



# https://www.kaggle.com/moltean/fruits
train_path = 'Train_horse_human/'
valid_path = 'Validation_horse_human/'


# useful for getting number of files
image_files = glob(train_path + '/*/*.jp*g')
valid_image_files = glob(valid_path + '/*/*.jp*g')

# useful for getting number of classes
folders = glob(train_path + '/*')

In [6]:
!pip install tensorflow

     |████████████████████████████████| 320.4 MB 14 kB/s s eta 0:00:011     |███████████████████████▊        | 237.2 MB 115.3 MB/s eta 0:00:01     |████████████████████████████▋   | 286.0 MB 98.7 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 96.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 53.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 516 kB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 121 kB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 87.7 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 92.5 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 93.1 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 188 kB/s  eta 0:00:01
     |████████████████████████████████| 88 kB 423 kB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 90.4 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 100.3 MB/s eta 0:00:01
  Created wheel for termcolor: 

In [7]:
import tensorflow as tf

In [9]:
# create an instance of ImageDataGenerator
gen = tf.keras.preprocessing.image.ImageDataGenerator(
  rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.2,
  horizontal_flip=True,
  vertical_flip=True,
  preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

train_generator = gen.flow_from_directory(
  train_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
)
valid_generator = gen.flow_from_directory(
  valid_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [10]:
#VGG16 Example

#include_top = True ----> You are going to use the entire pre-trained model which is trained in Imgnet.
#include_top = False

vgg = tf.keras.applications.vgg16.VGG16(input_shape=[100,100,3] ,
                                       weights="imagenet",
                                       include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [11]:
#Dont train existing weights  -- Locking the pretrained weights
for layer in vgg.layers:
    layer.trainable = False

In [12]:
#Adding our layers -- Fully Connected
flatten = tf.keras.layers.Flatten()
#...
finalLayer = tf.keras.layers.Dense(7, activation="softmax")

model = tf.keras.models.Sequential([
    vgg,  #vgg without fully connected network
    flatten, #Adding Flattening
    finalLayer #Adding Output Layer
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 32263     
Total params: 14,746,951
Trainable params: 32,263
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
#Compile
model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [15]:
#Fit
history = model.fit(train_generator,
                    validation_data=valid_generator,
                   epochs=5,
                   steps_per_epoch= len(train_generator.filenames) // batch_size,
                   validation_steps= len(valid_generator.filenames) // batch_size,
                   use_multiprocessing=False)

Epoch 1/5


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,7] labels_size=[32,2]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-15-74e39948cc5a>:7) ]] [Op:__inference_train_function_1334]

Function call stack:
train_function
